# Analysis of Irish Arts Council Funding, 2003 - 2017  

This notebook aims to analyse the funding awarded by the Arts Council of Ireland 
to artists working in a range of artforms over the period 2003 to 2017.

In [1]:
# Import libraries. Pandas requires xlrd to be installed, to handle older 
# Excel files. 
import pandas as pd

## Data Source  
**Arts Council Grants to Artists by Artform, 2003-2010, 2016, 2017.**  
- The Arts Council does not publish historical funding data but the above breakdowns are available from the [Central Statistics Office](www.cso.ie), via CSO publications on ["Women and Men in Ireland"](https://www.cso.ie/en/statistics/womenandmeninireland/).  
(2013 publication does not contain any Arts Council data.)  
- **Note: the date in the URL refers to the date of publication, not the period of the data.**
- These tables do not represent all grants made by the Arts Council, but only grants 
made to individual artists (via schemes such as Cnuas, Artists’ Bursaries, Artists’ Awards.)

## Reading in data

In [2]:
# Try Excel import using pandas.read_excel()
# https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
# https://www.geeksforgeeks.org/python/working-with-excel-files-using-pandas/


URL_2003 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/otherreleases/2004/entiretablesandgraphs.xls"
URL_2004 = "https://www.cso.ie/en/media/csoie/releasespublications/Women_and_Men_in_Ireland_2005_-_Tables_and_Graphs.xls"
URL_2005 = "https://www.cso.ie/en/media/csoie/releasespublications/Women_and_Men_in_Ireland_2006_-_Tables_and_Graphs.xls"
URL_2006 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/otherreleases/2007/womenandmen2007.xls"
URL_2007 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/otherreleases/2008/womenandmen2008.xls"
URL_2008 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/otherreleases/2009/womenandmen2009.xls"
URL_2009 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/otherreleases/2010/womenandmen2010.xls"
URL_2010 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/otherreleases/2011/womenandmen2011.xls"
URL_2016 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/ep/womenandmeninireland/2016/socialcohesionandlifestyles/P-WAMII2016TBL3.17.xlsx"
URL_2017 = "https://www.cso.ie/en/media/csoie/releasespublications/documents/ep/womenandmeninireland/2019/genderequality/P-WAMII2019TBL4.9.xlsx"

column_names = ["Artform", ("Recipients","Male"), ("Recipients","Female"), ("Avg_Grant","Male"), ("Avg_Grant","Female")]
column_dtypes = {0:str, 1:int, 2:int, 3:float, 4:float} 

df_2003 = pd.read_excel(URL_2003, sheet_name="Table 4.11", header=[3], usecols="A:E", index_col=0, nrows=10, names=column_names, dtype=column_dtypes)
df_2004 = pd.read_excel(URL_2004, sheet_name="Table 4.13", header=[2], usecols="A:E", index_col=0, nrows=11, names=column_names, dtype=column_dtypes)
df_2005 = pd.read_excel(URL_2005, sheet_name="Table 4.13", header=[3], usecols="A:E", index_col=0, nrows=13, names=column_names, dtype=column_dtypes)
df_2006 = pd.read_excel(URL_2006, sheet_name="Table 4.12", header=[3], usecols="A:E", index_col=0, nrows=13, names=column_names, dtype=column_dtypes)
df_2007 = pd.read_excel(URL_2007, sheet_name="Table 4.12", header=[3], usecols="A:E", index_col=0, nrows=12, names=column_names, dtype=column_dtypes)
df_2008 = pd.read_excel(URL_2008, sheet_name="Table 3.21", header=[3], usecols="A:E", index_col=0, nrows=12, names=column_names, dtype=column_dtypes)
df_2009 = pd.read_excel(URL_2009, sheet_name="Table 3.21", header=[3], usecols="A:E", index_col=0, nrows=10, names=column_names, dtype=column_dtypes)
df_2010 = pd.read_excel(URL_2010, sheet_name="Table 3.21", header=[3], usecols="A:E", index_col=0, nrows=11, names=column_names, dtype=column_dtypes)
df_2016 = pd.read_excel(URL_2016, header=[2], usecols="A:E", index_col=0, nrows=13, names=column_names, dtype=column_dtypes)
df_2017 = pd.read_excel(URL_2017, header=[2], usecols="A:E", index_col=0, nrows=14, names=column_names, dtype=column_dtypes)


# Look at one of the downloaded DataFrames
df_2016

Recipients        Avg_Grant         
                                           Male Female      Male   Female
Artform                                                                  
Architecture                                  6      3    4531.0   7140.0
Arts Participation                            4     10    4834.0  10668.0
Circus                                        7      4    6239.0  10559.0
Dance                                         5     27   18625.0  13486.0
Film                                          7      5   21133.0  30910.0
Literature                                   66     47   13787.0  10227.0
Music                                        55     30    9937.0   5952.0
Opera                                         6      4    8000.0   6688.0
Street Arts                                   1      1   19750.0    800.0
Theatre                                      18     29   18400.0  24751.0
Traditional Arts                             17     11    6008.0  13730.0
Visual Arts                                  79    103   13802.0  11987.0
Young people, children and education          2     17   10000.0   6059.0

In [3]:
# Calculate total grant for each artform  - start with example year
df_2005["Total_Grant"] = (
    df_2005["Recipients"]["Male"] * df_2005["Avg_Grant"]["Male"] 
    + df_2005["Recipients"]["Female"] * df_2005["Avg_Grant"]["Female"])

# show results
df_2005["Total_Grant"]

Artform
Architecture                  19600.0
Theatre                      163940.0
Dance                        167666.0
Drama                        228087.0
Education                     19123.0
Film/Film & Audio Visual     274610.0
Literature                   891599.0
Multidisciplinary arts       217904.0
Music                        635791.0
Opera                         47570.0
Visual arts                 1230076.0
Traditional Arts              35134.0
Other                         83539.0
Name: Total_Grant, dtype: float64

In [4]:
# Trying the df.product() method instead
# https://www.w3schools.com/python/pandas/ref_df_product.asp

df_2005[:]["Male"].product(axis="columns") + df_2005[:]["Female"].product(axis="columns")

Artform
Architecture                  19600.0
Theatre                      163940.0
Dance                        167666.0
Drama                        228087.0
Education                     19123.0
Film/Film & Audio Visual     274610.0
Literature                   891599.0
Multidisciplinary arts       217904.0
Music                        635791.0
Opera                         47570.0
Visual arts                 1230076.0
Traditional Arts              35134.0
Other                         83539.0
dtype: float64

## To Do:  
- read into a more useful data structure
- look at comparative data sources - Eurostat? Arts Council annual grant figures?